# Pré-Processamento: SparkSQL

### Step 1: Start Spark Session

In [9]:
import os
import sys

# Limpar ambiente Spark completamente
def reset_spark():
    for var in ['spark', 'sc', 'sqlContext']:
        if var in globals():
            try:
                del globals()[var]
            except:
                pass
    
    # Limpar módulos
    modules = [k for k in sys.modules.keys() if 'pyspark' in k]
    for m in modules:
        if m in sys.modules:
            del sys.modules[m]

reset_spark()

from pyspark.sql import SparkSession

try:
    spark.stop()
except NameError:
    print("SparkContext not defined")

# local mode
# spark = SparkSession.builder \
#             .appName("Spark SQL") \
#             .master("local[*]") \
# 	    	.config("spark.jars.packages", "org.xerial.snappy:snappy-java:1.1.10.1") \
# 	    	.getOrCreate()

# cluster mode
spark = SparkSession.builder \
           .appName("Spark SQL basic example") \
           .master("spark://spark-master-g7:7077") \
	    	.config("spark.some.config.option", "some-value") \
	    	.getOrCreate()

25/07/09 02:04:04 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/07/09 02:04:04 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:295)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:955)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:166)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:273)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:174)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:116)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:216)
	at org.apache.spark.rpc.netty.Inbox.proce

In [2]:
from pathlib import Path
import gdown
import zipfile

ROOT_DIR = Path.cwd().parent
data_path = ROOT_DIR / "data"
data_raw_path = data_path / "data_raw"
zip_path = data_path / "data_raw.zip"
file_id = "1wLFeP8SPEuq_Ac6cPZTFE4PlR3bYdMRP"
download_url = f"https://drive.google.com/uc?export=download&id={file_id}"

if not data_path.exists() or not data_raw_path.exists():
    print(f"Pasta {data_raw_path} não encontrada. Baixando os dados...")
    
    data_path.mkdir(parents=True, exist_ok=True)

    gdown.download(download_url, str(zip_path), quiet=False)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_path)

    zip_path.unlink()

    print("Download e extração concluídos.")
else:
    print(f"Pasta {data_raw_path} encontrada.")

Pasta /app/data/data_raw não encontrada. Baixando os dados...


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1wLFeP8SPEuq_Ac6cPZTFE4PlR3bYdMRP
From (redirected): https://drive.google.com/uc?export=download&id=1wLFeP8SPEuq_Ac6cPZTFE4PlR3bYdMRP&confirm=t&uuid=32333113-c95f-4991-8882-e96ed46bec59
To: /app/data/data_raw.zip
100%|██████████| 1.41G/1.41G [00:54<00:00, 25.8MB/s]


Download e extração concluídos.


In [3]:
import glob

arquivos_detalhados = glob.glob("../data/data_raw/**/*_DETAIL_*.csv", recursive=True)

arquivos_principais = [
    arq for arq in glob.glob("../data/data_raw/**/*.csv", recursive=True)
    if "_DETAIL_" not in arq
]

In [ ]:
# arquivos_detalhados = "../datasample/RESTRICAO_COFF_EOLICA_DETAIL_2025_05_amostra.csv"

# arquivos_principais = "../datasample/RESTRICAO_COFF_EOLICA_2025_05_amostra.csv"

In [4]:
df_detalhado = spark.read.option("delimiter", ";").option("header", True).option("inferSchema", True).csv(arquivos_detalhados)

df_principal = spark.read.option("delimiter", ";").option("header", True).option("inferSchema", True).csv(arquivos_principais)

In [5]:
df_detalhado.show()
df_detalhado.count()

+-------------+---------+----------------------+-------------------+------------+------+--------------------+-------------------+-------------------+---------------------+-------------------+---------------------+
|id_subsistema|id_estado|nom_modalidadeoperacao|  nom_conjuntousina|   nom_usina|id_ons|                 ceg|       din_instante|val_ventoverificado|flg_dadoventoinvalido|val_geracaoestimada|val_geracaoverificada|
+-------------+---------+----------------------+-------------------+------------+------+--------------------+-------------------+-------------------+---------------------+-------------------+---------------------+
|            N|       MA|             Tipo II-C|Conj. Paulino Neves|   Delta 3 I|MAEDT1|EOL.CV.MA.033682-...|2022-05-01 00:00:00|              1.684|                  0.0|               0.25|                0.585|
|            N|       MA|             Tipo II-C|Conj. Paulino Neves|  Delta 3 II|MAEDT2|EOL.CV.MA.033683-...|2022-05-01 00:00:00|              1

55577477

In [6]:
df_principal.show()
df_principal.count()

+-------------+--------------+---------+----------+-------------------+---------+---+-------------------+-----------+-------------------+-------------------+---------------------+--------------------------+------------------+-------------------+
|id_subsistema|nom_subsistema|id_estado|nom_estado|          nom_usina|   id_ons|ceg|       din_instante|val_geracao|val_geracaolimitada|val_disponibilidade|val_geracaoreferencia|val_geracaoreferenciafinal|cod_razaorestricao|cod_origemrestricao|
+-------------+--------------+---------+----------+-------------------+---------+---+-------------------+-----------+-------------------+-------------------+---------------------+--------------------------+------------------+-------------------+
|            N|         NORTE|       MA|  MARANHAO|CONJ. PAULINO NEVES|CJU_MAPLN|  -|2024-08-01 00:00:00|    156.526|               NULL|              396.4|              169.492|                      NULL|              NULL|               NULL|
|            N| 

9503712

In [7]:
df_detalhado.write.option("compression", "uncompressed").mode("overwrite").parquet("../data/data_cleared/dados_detalhados.parquet")

In [8]:
df_principal.write.option("compression", "uncompressed").mode("overwrite").parquet("../data/data_cleared/dados_principais.parquet")

In [ ]:
# Aplicar transformações aos dados principais antes de salvar
from pyspark.sql.functions import col, when, year, month, hour

print("Aplicando transformações aos dados...")

# Calcular constrained-off
df_principal = df_principal.withColumn(
    "constrained_off",
    when(col("val_disponibilidade") - col("val_geracao") > 0,
         col("val_disponibilidade") - col("val_geracao"))
    .otherwise(0)
)

# Calcular percentual de constrained-off
df_principal = df_principal.withColumn(
    "percentual_constrained",
    when(col("val_disponibilidade") > 0,
         (col("constrained_off") / col("val_disponibilidade")) * 100)
    .otherwise(0)
)

# Adicionar colunas temporais
df_principal = df_principal.withColumn("ano", year("din_instante"))
df_principal = df_principal.withColumn("mes", month("din_instante"))
df_principal = df_principal.withColumn("hora", hour("din_instante"))

print("✅ Transformações aplicadas")
print("Colunas adicionadas: constrained_off, percentual_constrained, ano, mes, hora")

In [ ]:
### 📊 Para Análises de Performance
**Nota**: Para análises comparativas de performance entre CSV e Parquet e benchmarking completo, utilize o notebook `Performance-Benchmark.ipynb`.